In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY5")

In [5]:
print(KEY)

sk-proj-r1BnlOwScc3fykuSQ2mYT3BlbkFJF8yPW33GXT4dG57qOTWm


In [6]:
llm=ChatOpenAI(openai_api_key=KEY , model_name="gpt-3.5-turbo" , temperature=0.5)

c:\Users\zuhai\OneDrive\Desktop\PROJECT2\MCQ_GENERATOR\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001B23CEE0310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001B23CEEE190>, temperature=0.5, openai_api_key='sk-proj-r1BnlOwScc3fykuSQ2mYT3BlbkFJF8yPW33GXT4dG57qOTWm', openai_proxy='')

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text: {text}

You are an expert MCQ maker. Given the above text, it is your job to \

create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.

Make sure the questions are not repeated and check all the questions to be conforming the text as well. Make

sure to format your response like RESPONSE_JSON below and use it as a guide. \

Ensure to make {number} MCQs

### RESPNSE_JSON
{response_json}
"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text" , "number" , "subject" , "tone" , "response_json"],
    template=TEMPLATE
)

In [12]:
quiz_chain=LLMChain(llm=llm , prompt=quiz_generation_prompt , output_key="quiz" , verbose=True)

c:\Users\zuhai\OneDrive\Desktop\PROJECT2\MCQ_GENERATOR\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject" , "quiz"] , template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm , prompt=quiz_evaluation_prompt , output_key="review" , verbose=True )

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain , review_chain] , input_variables=["text" , "number" , "subject" , "tone" , "response_json"],output_variables=["quiz" , "review"] , verbose=True)

In [17]:
file_path=r"C:\Users\zuhai\OneDrive\Desktop\PROJECT2\MCQ_GENERATOR\data.txt"

In [18]:
with open(file_path , 'r') as file:
    TEXT = file.read()

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [21]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\zuhai\OneDrive\Desktop\PROJECT2\MCQ_GENERATOR\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used

In [22]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2592
Prompt Tokens:1796
Completion Tokens:796
Total Cost:0.004286


In [23]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [45]:
quiz=response.get("quiz")

In [39]:
print(response.get("quiz"))


{
  "1": {
    "mcq": "Who coined the term 'machine learning' in 1959?",
    "options": {
      "a": "Donald Hebb",
      "b": "Tom M. Mitchell",
      "c": "Arthur Samuel",
      "d": "Walter Pitts"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
    "options": {
      "a": "Cybertron",
      "b": "Deep Blue",
      "c": "HAL 9000",
      "d": "Skynet"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "Who proposed the early mathematical models of neural networks for machine learning algorithms?",
    "options": {
      "a": "Donald Hebb",
      "b": "Warren McCulloch",
      "c": "Tom M. Mitchell",
      "d": "Arthur Samuel"
    },
    "correct": "b"
  },
  "4": {
    "mcq": "What is the main objective of modern-day machine learning?",
    "options": {
      "a": "To confuse data",
      "b": "To make predictions based on models",
      "c": "To hide data",
      "d": "T

In [46]:
quiz=json.loads(quiz)

In [47]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [48]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybertron | b: Deep Blue | c: HAL 9000 | d: Skynet',
  'Correct': 'a'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks for machine learning algorithms?',
  'Choices': 'a: Donald Hebb | b: Warren McCulloch | c: Tom M. Mitchell | d: Arthur Samuel',
  'Correct': 'b'},
 {'MCQ': 'What is the main objective of modern-day machine learning?',
  'Choices': 'a: To confuse data | b: To make predictions based on models | c: To hide data | d: To ignore data',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, when does a computer program learn from experience?',
  'Choices': 'a: When it stops working | b: When its performance improves with experience | c: When it forg

In [53]:
quiz=pd.DataFrame(quiz_table_data)

In [54]:
quiz.to_csv("machinelearning.csv",index=False)